# Botnet on Twitter?

In [1]:
import pandas
import igraph
import graphistry
graphistry.register(key='MY_API_KEY') #https://www.graphistry.com/api-request

## Step 1: Loading The Data

This dataset was created by a Twitter user who was surprised that one of his very innocuous tweet ("Hey let's grab a coffee") got retweeted several times. Intrigued, he had a closer look at the accounts that retweeted his message. He found that those accounts all had inprononcable names that looked like gibberish. Suspecting that those accounts might be fake, he crawled the twitter social network around the suspicious accounts to produce this dataset.

The dataset is in a CSV file named `twitterDemo.csv` which looks like that:
```
#dstAccount,srcAccount
arley_leon16,wxite_pymp
michaelinhooo2,wxite_pymp
steeeva,wxite_pymp
...
```
Each row in `twitterDemo.csv` denotes two twitter accounts "following" (Twitter's equivalent of friending) each other.
- - - 
1. Use Pandas to load the CSV and print the first three lines:
```python
follows = pandas.read_csv('twitterDemo.csv')
follows[:3]
```

In [2]:
follows = pandas.read_csv('../data/twitterDemo.csv')
follows[:3]

,dstAccount,srcAccount
0,arley_leon16,wxite_pymp
1,michaelinhooo2,wxite_pymp
2,steeeva,wxite_pymp


## Step 2: First Simple Visualization

We can visualize this subset of the Twitter network as a graph: Each node is a Twitter account and edges encode the "follows" relation.

- - - 
1. To define the graph, we bind *source* and *destination* to the columns indicating the start and end nodes of each edges.
```python
plotter = graphistry.bind(source='srcAccount', destination='dstAccount')
```
2. Then we call plot to create the visualization
```python
plotter.plot(follows)
```

In [3]:
plotter = graphistry.bind(source='srcAccount', destination='dstAccount')
plotter.plot(follows)

Can you answer the following questions by exploring the visualization you have just created?
- Is the structure of the graph what you would expect from a social network?
- Can you tell which accounts might be fake and which ones are likely real users?

## Step 3: Computing Graph Metrics With IGraph

Next, we are going to use [IGraph](http://igraph.org/python/), a graph computation library, to compute metrics like pagerank to help us understand the dataset.

---
1. First we convert our data to an Igraph
```python
ig = plotter.pandas2igraph(follows)
igraph.summary(ig)
```
2. Then we compute centrality metrics. Each metric is stored as a node or edge attribute
```python
ig.vs['pagerank'] = ig.pagerank(directed=False)
ig.vs['betweenness'] = ig.betweenness(directed=True)
ig.es['ebetweenness'] = ig.edge_betweenness(directed=True)
```
3. Finally, we run three community detection algorithm
```python
ig.vs['community_spinglass'] = ig.community_spinglass(spins=12, stop_temp=0.1, cool_fact=0.9).membership
uig = ig.copy()
uig.to_undirected()
ig.vs['community_infomap'] = uig.community_infomap().membership
ig.vs['community_louvain'] = uig.community_multilevel().membership
```
4. We can now bind visual properties such as node size and color to graph attributes. Let's size nodes based on their `pagerank` score and color them using `community_spinglass`
```python
plotter2 = plotter.bind(point_color='community_spinglass', point_size='pagerank')
```
5. We are now ready to create a new visualization:
```python
plotter2.plot(ig)
```

In [4]:
ig = plotter.pandas2igraph(follows)
igraph.summary(ig)

IGRAPH D--- 7889 10063 -- 
+ attr: __nodeid__ (v)


In [5]:
ig.vs['pagerank'] = ig.pagerank(directed=False)
ig.vs['betweenness'] = ig.betweenness(directed=True)
ig.es['ebetweenness'] = ig.edge_betweenness(directed=True)

In [6]:
ig.vs['community_spinglass'] = ig.community_spinglass(spins=12, stop_temp=0.1, cool_fact=0.9).membership
uig = ig.copy()
uig.to_undirected()
ig.vs['community_infomap'] = uig.community_infomap().membership
ig.vs['community_louvain'] = uig.community_multilevel().membership

In [7]:
plotter2 = plotter.bind(point_color='community_spinglass', point_size='pagerank')
plotter2.plot(ig)

## Step 4: Visual Drill Downs

Within the visualization, you can filter and drill down into the graph. Try the following:

1. Open the histogram panel, and add histograms for `pagerank`, `betweenness`, `ebetweenness`, etc. By selecting a region of a histogram or clicking on a bar, you can filter the graph.

2. You can also manually create filters in the filter panel ("funnel" icon in the left menu bar). For instance, try filtering on `point:pagerank` such that `point:pagerank >= 0.01`. We select the most "influencial accounts". Those are the likely botnet owners/customers.

3. Still in the histogram panel, you can visually show attributes using on the graph node/edge colors. Try clicking on each of the three square icons on top of each histogram. Notice that when point color is bound to `community_spinglass`, the "tail" of the network forms a distinct community. What makes those accounts different from the rest?

4. With the histogram panel open, click on data brush and then lasso a selection on the graph. The histograms highlight the subset of nodes under the selection. You can drag the data brush selection to compare different subgraphs.
